## 1.Overview of Semi-supervised Learning

### 1.Active Learning
--- Need extra label information from expert
#### a. Train a model with labeled data
#### b. Pick one unlabeled data, compare result with the result gained from the expert
#### c. Retrain the model with the information

### 2.Pure transductive Learning
--- No extra label from the expert
#### a. Train a model with both labeled and unlabeled data  
#### b. Use the model to predict new data

### 3.Transductive Learning
--- No extra label from the expert\n
#### a. Train a model with both labeled data  
#### b. Use the model to label unlabeled data

## 2. Models of semi-superviesed learning

#### 1.Generative methods 
EM methods --- need to know the distribution of the whole data

#### 2.Semi-Supervised Support Vector Machine S3VM
TSVM --- the objective function is not convex 

#### 3.Graph Semi-Supervised Learning
Label Propagation --- the quality of the graph matters

#### 4.Disagreement-based methods
Co-training --- combine with Ensemble Learning and  based on the design of basic learning machine

#### 5.More
Manifold regularization; S4VM (safe semi-supervised Learning)

## 3. Data prepartion for the models

In [142]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.svm as svm
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score
import warnings

warnings.filterwarnings('ignore')

In [167]:
from __future__ import print_function
from __future__ import division

In [143]:
## display all the columns
pd.options.display.max_columns = 120
pd.options.display.max_rows = 100

In [144]:
## one hot encoding methods
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# define example

def onehotencoding_fun(data):
    values = array(data)
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #print(onehot_encoded)
    # invert first example
    inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
    #print(inverted)
    return onehot_encoded 

In [159]:
df=pd.read_csv('data_onehot.csv')

In [160]:
df['J_label'] = 0.0
df['J_label'][0:10] = 1
df['J_label'][20:25] = 1
df['J_label'][36:41] = 1
df['J_label'][50:60] = 1
df['J_label'][10:20] = -1
df['J_label'][25:36] = -1
df['J_label'][41:50] = -1
df

,Unnamed: 0,author,contents,description,publisher,source_url,title,date,time,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,Average,Volatility,SD,publisherLabel,Mark,publisher_L,author_L,score_sentiment,magnitude_sentiment,tfidf,publisher_L_onehot,author_L_onehot,J_label
0,662,Thomas Delahunty,bitcoin news price information analysis almost...,almost ten years since blockchain technology u...,Newsbtc.com,https://www.newsbtc.com/2018/02/05/israel-beco...,"Israel Becoming “Crypto Powerhouse,” But ICO R...",2/5/18,0:00:17,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,723.0,-1.0,3.0,2.0,-0.2,0.2,0.29965,0,0,1.0
1,664,The Star Online,advertisement monday feb across lloyds bank ba...,lloyds banking group plc said sunday would ban...,Thestar.com.my,http://www.thestar.com.my/business/business-ne...,Lloyds Bank to ban credit card owners from buy...,2/5/18,0:26:03,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,1056.0,-1.0,2.0,2.0,0.0,0.0,0.34320,0,0,1.0
2,665,Mydealz.de,kann man der blase sch n beim platzen zuschaue...,itunes aktuell gibt es die app crypto pro bitc...,Mydealz.de,https://www.mydealz.de/deals/crypto-pro-bitcoi...,Crypto Pro: Bitcoin Ticker kostenlos für iOS,2/5/18,0:39:15,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,695.0,-1.0,0.0,0.0,0.7,0.7,0.21495,0,0,1.0
3,667,JTMusic,added feb jtmusic tags someone left messege bi...,bought bitcoin peak k dropping k hype gone sun,Liveleak.com,https://www.liveleak.com/view?i=f9d_1517791670,Someone left me a messege about Bitcoin,2/5/18,0:50:37,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,621.0,-1.0,0.0,0.0,0.1,0.1,0.65890,0,0,1.0
4,668,marcuss,one persistent yet baffling narratives frames ...,one persistent yet baffling narratives frames ...,Valuewalk.com,http://www.valuewalk.com/2018/02/cryptocurrenc...,The Worst Argument Against Cryptocurrencies,2/5/18,0:52:53,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,1117.0,-1.0,3.0,0.0,-0.9,0.9,0.35514,0,0,1.0
5,669,Thomas Delahunty,bitcoin news price information analysis speaki...,speaking cnbc nfl star richard sherman cornerb...,Newsbtc.com,https://www.newsbtc.com/2018/02/05/richard-she...,Richard Sherman: Bitcoin Becoming a Common Con...,2/5/18,1:00:13,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,723.0,-1.0,3.0,2.0,0.0,0.0,0.30933,0,0,1.0
6,673,CoinTelegraph By Vlad Smerkis,cointelegraph indices cryptocurrencies perform...,bitcoin cash dash vs bse sensex hang seng inde...,Cointelegraph.com,https://cointelegraph.com/news/equity-markets-...,Equity Markets vs. Cryptocurrency Markets: Wee...,2/5/18,1:29:51,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,199.0,-1.0,3.0,0.0,0.6,0.6,0.72590,0,0,1.0
7,675,Adam James,real time prices vires numeris bitcoin ethereu...,bitcoin cryptocurrencies tips many people tong...,Bitcoinist.com,http://bitcoinist.com/nfl-players-interested-i...,"NFL Players Interested in Tackling Bitcoin, Se...",2/5/18,1:45:19,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,96.0,-1.0,3.0,3.0,0.0,0.0,0.31311,0,0,1.0
8,676,The Quint,growing number big us credit card issuers deci...,business news today bitcoin ban grows govt for...,Thequint.com,https://www.thequint.com/news/business/busines...,Business News Today: Bitcoin Ban Grows; Govt t...,2/5/18,1:55:07,7615.109937,7629.812951,7598.931764,7613.970403,41.047748,297975.9020,7614.280886,11917.267390,446037.8161,2146.000443,1045.0,-1.0,0.0,0.0,0.3,0.3,0.57525,0,0,1.0
9,677,Ilias Louis Hatzis,block

In [161]:
data = df[['publisher_L_onehot','author_L_onehot','score_sentiment','magnitude_sentiment','tfidf']]
label = df['J_label']

## 4. Codes for different models

## 1.Transductive support vector machines

In [162]:
## from 周志华 机器学习西瓜书
def TSVM_fun(l_d,l_c,u_d):
    lu_d = np.concatenate((l_d, u_d))
    n = len(l_d)+len(u_d)
    
    ## initialize SVM
    clf1 = svm.SVC(C=1,kernel='linear')
    clf1.fit(l_d, l_c)
    clf0 = svm.SVC(C=1,kernel='linear')
    clf0.fit(l_d, l_c)
    lu_c_0 = clf0.predict(lu_d) 
    
    # the pseudo label for unlabelled samples
    u_c_new = clf1.predict(u_d)  
    cu, cl = 0.001, 1
    sample_weight = np.ones(n)
    sample_weight[len(l_c):] = cu
    id_set = np.arange(len(u_d))
    
    while cu < cl:
        lu_c = np.concatenate((l_c, u_c_new))  # 70
        clf1.fit(lu_d, lu_c, sample_weight=sample_weight)
        while True:
            u_c_new = clf1.predict(u_d)  # the pseudo label for unlabelled samples
            u_dist = clf1.decision_function(u_d)  # the distance of each sample
            norm_weight = np.linalg.norm(clf1.coef_)  # norm of weight vector
            epsilon = 1 - u_dist * u_c_new * norm_weight

            plus_set, plus_id = epsilon[u_c_new > 0], id_set[u_c_new > 0]  # positive labelled samples
            minus_set, minus_id = epsilon[u_c_new < 0], id_set[u_c_new < 0]  # negative labelled samples
            plus_max_id, minus_max_id = plus_id[np.argmax(plus_set)], minus_id[np.argmax(minus_set)]
            a, b = epsilon[plus_max_id], epsilon[minus_max_id]

            if a > 0 and b > 0 and a + b > 2:
                u_c_new[plus_max_id], u_c_new[minus_max_id] = -u_c_new[plus_max_id], -u_c_new[minus_max_id]
                lu_c = np.concatenate((l_c, u_c_new))
                clf1.fit(lu_d, lu_c, sample_weight=sample_weight)
            else:
                break
        cu = min(cu * 2, cl)
        sample_weight[len(l_c):] = cu
    lu_c = np.concatenate((l_c, u_c_new))
    return lu_c

In [163]:
l_d, l_c = data[0:60], label[0:60] 
u_d = data[60:]
lu_c = TSVM_fun(l_d, l_c, u_d)
lu_c

array([ 1.,  1.,  1., ...,  1., -1., -1.])

## 2.Label propagation

In [165]:
# coding=utf8
"""
Graph-Based Semi-Supervised Learning (GBSSL) implementation.
"""

# Authors: Yuto Yamaguchi <yamaguchi.yuto@aist.go.jp>
# Lisence: MIT

import numpy as np
from scipy import sparse
from abc import ABCMeta, abstractmethod
from sklearn.base import BaseEstimator, ClassifierMixin

class Base(BaseEstimator, ClassifierMixin):
    __metaclass__ = ABCMeta
    def __init__(self,graph,max_iter=30):
        self.max_iter = max_iter
        self.graph = graph

    @abstractmethod
    def _build_propagation_matrix(self):
        raise NotImplementedError("Propagation matrix construction must be implemented to fit a model.")

    @abstractmethod
    def _build_base_matrix(self):
        raise NotImplementedError("Base matrix construction must be implemented to fit a model.")

    def _init_label_matrix(self):
        n_samples = self.graph.shape[0]
        n_classes = self.y_.max()+1
        return np.zeros((n_samples,n_classes))

    def _arrange_params(self):
        """Do nothing by default"""
        pass

    def fit(self,x,y):
        """Fit a graph-based semi-supervised learning model
        All the input data is provided array X (labeled samples only)
        and corresponding label array y.
        Parameters
        ----------
        x : array_like, shape = [n_labeled_samples]
            Node IDs of labeled samples
        y : array_like, shape = [n_labeled_samples]
            Label IDs of labeled samples
        Returns
        -------
        self : returns an instance of self.
        """
        self.x_ = x
        self.y_ = y

        self._arrange_params()

        self.F_ = self._init_label_matrix()

        self.P_ = self._build_propagation_matrix()
        self.B_ = self._build_base_matrix()

        remaining_iter = self.max_iter
        while remaining_iter > 0:
            self.F_ = self._propagate()
            remaining_iter -= 1

        return self

    def _propagate(self):
        return self.P_.dot(self.F_) + self.B_

    def predict(self,x):
        """Performs prediction based on the fitted model
        Parameters
        ----------
        x : array_like, shape = [n_samples]
            Node IDs
        Returns
        -------
        y : array_like, shape = [n_samples]
            Predictions for input node IDs
        """
        probas = self.predict_proba(x)
        return np.argmax(probas,axis=1)

    def predict_proba(self,x):
        """Predict probability for each possible label
        Parameters
        ----------
        x : array_like, shape = [n_samples]
            Node IDs
        Returns
        -------
        probabilities : array_like, shape = [n_samples, n_classes]
            Probability distributions across class labels
        """
        z = np.sum(self.F_[x], axis=1)
        z[z==0] += 1 # Avoid division by 0
        return (self.F_[x].T / z).T


class LGC(Base):
    """Local and Global Consistency (LGC) for GBSSL
    Parameters
    ----------
    alpha : float
      clamping factor
    max_iter : float
      maximum number of iterations allowed
    Attributes
    ----------
    x_ : array, shape = [n_samples]
        Input array of node IDs.
    References
    ----------
    Zhou, D., Bousquet, O., Lal, T. N., Weston, J., & Schölkopf, B. (2004).
    Learning with local and global consistency.
    Advances in neural information processing systems, 16(16), 321-328.
    """

    def __init__(self,graph=None,alpha=0.99,max_iter=30):
        super(LGC, self).__init__(graph,max_iter=30)
        self.alpha=alpha

    def _build_propagation_matrix(self):
        """ LGC computes the normalized Laplacian as its propagation matrix"""
        degrees = self.graph.sum(axis=0).A[0]
        degrees[degrees==0] += 1  # Avoid division by 0
        D2 = np.sqrt(sparse.diags((1.0/degrees),offsets=0))
        S = D2.dot(self.graph).dot(D2)
        return self.alpha*S

    def _build_base_matrix(self):
        n_samples = self.graph.shape[0]
        n_classes = self.y_.max()+1
        B = np.zeros((n_samples,n_classes))
        B[self.x_,self.y_] = 1
        return (1-self.alpha)*B

class HMN(Base):
    """Harmonic funcsion (HMN) for GBSSL
    Parameters
    ----------
    max_iter : float
      maximum number of iterations allowed
    Attributes
    ----------
    x_ : array, shape = [n_samples]
        Input array of node IDs.
    References
    ----------
    Zhu, X., Ghahramani, Z., & Lafferty, J. (2003, August).
    Semi-supervised learning using gaussian fields and harmonic functions.
    In ICML (Vol. 3, pp. 912-919).
    """
    def __init__(self,graph=None,max_iter=30):
        super(HMN, self).__init__(graph,max_iter=30)

    def _build_propagation_matrix(self):
        degrees = self.graph.sum(axis=0).A[0]
        degrees[degrees==0] += 1  # Avoid division by 0
        D = sparse.diags((1.0/degrees),offsets=0)
        P = D.dot(self.graph).tolil()
        P[self.x_] = 0
        return P.tocsr()

    def _build_base_matrix(self):
        n_samples = self.graph.shape[0]
        n_classes = self.y_.max()+1
        B = np.zeros((n_samples,n_classes))
        B[self.x_,self.y_] = 1
        return B

class PARW(Base):
    """Partially Absorbing Random Walk (PARW) for GBSSL
    Parameters
    ----------
    lamb: float (default=0.001)
      Absorbing parameter
    max_iter : float
      maximum number of iterations allowed
    Attributes
    ----------
    x_ : array, shape = [n_samples]
        Input array of node IDs.
    References
    ----------
    Wu, X. M., Li, Z., So, A. M., Wright, J., & Chang, S. F. (2012).
    Learning with partially absorbing random walks.
    In Advances in Neural Information Processing Systems (pp. 3077-3085).
    """
    def __init__(self,graph=None,lamb=1.0,max_iter=30):
        super(PARW, self).__init__(graph,max_iter=30)
        self.lamb=lamb

    def _build_propagation_matrix(self):
        d = self.graph.sum(axis=1).T.A[0]
        Z = sparse.diags(1.0 / (d+self.lamb), offsets=0)
        P = Z.dot(self.graph)
        return P

    def _build_base_matrix(self):
        n_samples = self.graph.shape[0]
        n_classes = self.y_.max()+1
        B = np.zeros((n_samples,n_classes))
        B[self.x_,self.y_] = 1
        d = np.array(self.graph.sum(1).T)[0]
        Z = sparse.diags(1.0 / (d+self.lamb), offsets=0)
        Lamb = sparse.diags(self.lamb,shape=(n_samples,n_samples), offsets=0)
        return Z.dot(Lamb).dot(B)

class OMNI(Base):
    """OMNI-Prop for GBSSL
    Parameters
    ----------
    lamb : float > 0 (default = 1.0)
      Define importance between prior and evidence from neighbors
    max_iter : float
      maximum number of iterations allowed
    Attributes
    ----------
    x_ : array, shape = [n_samples]
        Input array of node IDs.
    References
    ----------
    Yamaguchi, Y., Faloutsos, C., & Kitagawa, H. (2015, February).
    OMNI-Prop: Seamless Node Classification on Arbitrary Label Correlation.
    In Twenty-Ninth AAAI Conference on Artificial Intelligence.
    """

    def __init__(self,graph=None,lamb=1.0,max_iter=30):
        super(OMNI,self).__init__(graph,max_iter)
        self.lamb = lamb

    def _build_propagation_matrix(self):
        d = self.graph.sum(axis=0).A[0]
        dT = self.graph.sum(axis=1).T.A[0]
        Q = (sparse.diags(1.0/(d+self.lamb), offsets=0).dot(self.graph)).dot(sparse.diags(1.0/(dT+self.lamb),offsets=0).dot(self.graph.T)).tolil()
        Q[self.x_] = 0
        return Q

    def _build_base_matrix(self):
        n_samples = self.graph.shape[0]
        n_classes = self.y_.max()+1
        unlabeled = np.setdiff1d(np.arange(n_samples),self.x_)

        dU = self.graph[unlabeled].sum(axis=1).T.A[0]
        dT = self.graph.sum(axis=0).A[0]
        n_samples = self.graph.shape[0]
        r = sparse.diags(1.0/(dU+self.lamb),offsets=0).dot(self.lamb*self.graph[unlabeled].dot(sparse.diags(1.0/(dT+self.lamb),offsets=0)).dot(np.ones(n_samples))+self.lamb)

        b = np.ones(n_classes) / float(n_classes)

        B = np.zeros((n_samples,n_classes))
        B[unlabeled] = np.outer(r,b)
        B[self.x_,self.y_] = 1
        return B

class CAMLP(Base):
    """Confidence-Aware Modulated Label Propagation (CAMLP) for GBSSL
    Parameters
    ----------
    beta : float > 0 (default = 0.1)
      Define importance between prior and evidence from neighbors
    H : array_like, shape = [n_classes, n_classes]
      Define affinities between labels
      if None, identity matrix is set
    max_iter : float
      maximum number of iterations allowed
    Attributes
    ----------
    x_ : array, shape = [n_samples]
        Input array of node IDs.
    References
    ----------
    Yamaguchi, Y., Faloutsos, C., & Kitagawa, H. (2016, May).
    CAMLP: Confidence-Aware Modulated Label Propagation.
    In SIAM International Conference on Data Mining.
    """

    def __init__(self,graph=None,beta=0.1,H=None,max_iter=30):
        super(CAMLP,self).__init__(graph,max_iter)
        self.beta=beta
        self.H=H

    def _arrange_params(self):
        if self.H is None:
            n_classes = self.y_.max()+1
            self.H = np.identity(n_classes)
        self.Z = self._build_normalization_term()

    def _propagate(self):
        return self.P_.dot(self.F_).dot(self.H) + self.B_

    def _build_normalization_term(self):
        d = self.graph.sum(axis=1).T.A[0]
        return sparse.diags(1.0/(1.0+d*self.beta),offsets=0)

    def _build_propagation_matrix(self):
        return self.Z.dot(self.beta*self.graph)

    def _build_base_matrix(self):
        n_samples = self.graph.shape[0]
        n_classes = self.y_.max()+1
        B = np.ones((n_samples,n_classes))/float(n_classes)
        B[self.x_] = 0
        B[self.x_,self.y_] = 1
        return self.Z.dot(B)

In [174]:
import numpy as np
import networkx as nx
from scipy import sparse
from label_propagation import LGC,HMN,PARW,OMNIProp,CAMLP
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split

G = nx.karate_club_graph()
labels = {'Officer':0, 'Mr. Hi':1}
nodes = np.array([(n,labels[attr['club']]) for n,attr in G.nodes(data=True)])
x = nodes[:,0]
y = nodes[:,1]
A = nx.to_scipy_sparse_matrix(G,nodelist=x)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.5,random_state=0)

methods = [('HMN', HMN(), {'graph':[A]}),
            ('LGC', LGC(), {'graph':[A], 'alpha':[0.01,0.05,0.1,0.5,0.99]}),
            ('PARW', PARW(), {'graph':[A], 'lamb':[0.01, 0.05, 0.01, 0.5, 0.99]}),
            ('OMNIProp', OMNIProp(), {'graph':[A], 'lamb':[0.01, 0.1, 1.0, 10.0, 100.0]}),
            ('CAMLP', CAMLP(), {'graph':[A], 'beta':[0.01, 0.1, 1.0, 10.0, 100.0], 'H':[np.array([[1,0],[0,1]]), np.array([[0,1],[1,0]])]})]

for name, clf, params in methods:
    print ("=========================")
    print (name)
    gs = GridSearchCV(clf, params, cv=5)
    gs.fit(x_train,y_train)

    print()
    print ("Grid Scores:")
    for score in gs.grid_scores_:
        print (score)

    model = gs.best_estimator_
    model.fit(x_train,y_train)
    print ("\nBest Estimator:")
    print (model)

    predicted = model.predict(x_test)
    acc = (predicted==y_test).mean()
    print ("\nAccuracy: %s" % acc)
    print()

ImportError: No module named 'label_propagation'